In [ ]:
import pandas as pd
from sklearn.preprocessing import RobustScaler

In [ ]:
import pandas as pd


data = pd.read_csv("Kor_income.csv")

#from google.colab import drive
#drive.mount('/content/drive')

#data = pd.read_csv("/content/drive/MyDrive/data/Kor_income.csv")
df = data.copy()
df

1. 수치형 컬럼으로 변환!

In [ ]:
import copy
new_df = copy.deepcopy(df) # deepcopy 를 이용하여 원본 데이터의 손상을 막음
for col in new_df.columns:
    new_df[col] = pd.to_numeric(new_df[col], errors='coerce')

new_df.head(10)

# 2. 관련없는 컬럼인 Unnamed: 0 삭제

In [ ]:
new_df.drop(columns='Unnamed: 0', axis=1, inplace=True)

In [ ]:
new_df

# 3. 이상치 삭제
- 삭제를 사용한 이유: 이상치들의 양이 전체 데이터의 약 0.1% 정도밖에 되지 않았기 때문에 이상치를 제거하는 것이 머신러닝의 정확도를 높여줄 것이라고 생각했음
- 다시 말해, 이상치가 있는 부분에서 특별한 의미를 찾기 힘들 것이라고 판단했음

In [ ]:
new_df.describe()

In [ ]:
import matplotlib.pyplot as plt
plt.boxplot(new_df['income'])

In [ ]:
new_df[new_df['income'] < 0]
# income이 0 미만인 경우는 161가지

In [ ]:
new_df.isna().sum()

In [ ]:
import matplotlib.pyplot as plt

plt.boxplot(new_df['religion'])

In [ ]:
len(new_df[new_df['religion'] > 2])
# religion에서 이상치는 2개. 전부 9로 기록되어있음

In [ ]:
plt.boxplot(new_df['marriage'])
# 이 중에서 컬럼 설명서에서는 나올 수 없는 9를 제거할 이상치로 보자!

In [ ]:
len(new_df[new_df['marriage']  == 9])
# marriage가 이상치인 경우는 7가지

In [ ]:
# 그러면 이제 이상치가 있는 경우들을 삭제하자.
new_df = new_df[new_df['income'] >= 0]
new_df = new_df[new_df['religion'] <= 2]
new_df = new_df[new_df['marriage'] < 9]
new_df = new_df[new_df['marriage'] != 0]
new_df.drop(new_df[new_df['company_size'] == 99].index,inplace=True)


new_df

In [ ]:
new_df.to_csv('new_df.csv')

In [ ]:
#reason_none_worker가 na면서 occupation이 na거나 company_size가 na인 값들을 제거함!
# ->돈을 벌지 않으면서 나머지 두 값 중 하나가 na라면 직업이 없고, 돈을 벌지 않는 상태라고 판단

new_df[new_df['reason_none_worker'].isna() & new_df['occupation'].isna()].dropna()
new_df[new_df['reason_none_worker'].isna() & new_df['company_size'].isna()].dropna()
new_df

In [ ]:
new_df[new_df['reason_none_worker'].isna()] = new_df[new_df['reason_none_worker'].isna()].fillna(0)
new_df[new_df['company_size'].isna()] = new_df[new_df['company_size'].isna()].fillna(0)
new_df[new_df['occupation'].isna()] = new_df[new_df['occupation'].isna()].fillna(0)
new_df

#앞서 언급한 경우를 제외하고 남아있는 Nan값을 0으로 설정

In [ ]:
#age 변수 추가
new_df['age'] = new_df['year'] - new_df['year_born'] + 1

In [ ]:
# age에 year과 yearborn을 이용하여 새로운 정보를 뽑아내었으니
# yearborn 컬럼 삭제 (year은 추후 물가 반영에 사용 예정)
# year과 wave는 동일하니 wave 역시 삭제
new_df.drop(columns={"year_born","wave"}, inplace=True)

In [ ]:
new_df.drop(new_df[(new_df['age'] < 18) & (new_df['marriage'] == 2)].index, inplace=True)

new_df

In [ ]:
new_df.corr()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(15,15))
sns.heatmap(data = new_df.corr(), annot=True,
fmt = '.2f', linewidths=.5, cmap='Blues')

#income과의 상관계수의 절댓값이 0.2 이상인 변수들을 주요 변수들로 보고 분석
# -> family_member, gender, education_level, company_size, reason_none_worker, age9

In [ ]:
# 머신에서 좀 더 정확하게 하기 위해 income을 기준으로 오름차순 정렬했음
new_df = new_df.sort_values(by='income')
new_df

In [ ]:
new_df.isna().sum()

### 추후 계획

- 1. 스케일러는 Robust Scaler를 사용할 예정: 나올 수 없는 이상치 외에는 크게 전처리를 하지 않았기에 이상치에 영향을 덜 받는 스케일러 중 하나인 Robust를 사용할 것임

- 2. 물가 반영률을 이용하여 income을 재조정 할 예정: 좀 더 정확한 예측 및 좀 더 많은 의미를 뽑아내기 위함. 아마 year를 뺄 수 있을 것이라고 판단됨.

- 3. 머신러닝에서 머신은 xgboost 를 사용할 예정: NA값을 0으로 처리한 케이스가 많아 xgboost를 사용하는 것이 적합해보임

In [ ]:
def replace_0_1(x):
    if int(x) == 0:
        x = 1
        return x
    else:
        return x
numeric_cols = ["income", "age", "family_member","company_size"]
for x in numeric_cols:
    new_df[x] = new_df[x].apply(replace_0_1)

In [ ]:
def log_transform(x):
    import numpy as np
    if x == 0:
        return x
    else:
        x = np.log(x)
        return x

numeric_cols = ["income", "age", "family_member","company_size"]
for x in numeric_cols:
    new_df[x] = new_df[x].apply(log_transform)

In [ ]:
new_df['income'].hist()

In [ ]:
#region encoding

def major_city(x):
    if x < 3:
        x = 1
        return x
    else:
        x = 0
        return x

new_df['major_city_enc'] = new_df['region'].apply(major_city)
new_df[['region', 'major_city_enc']]

In [ ]:
#marriage encoding (1 : couple, 0 : single)
def is_married(x):
    if(x == 2):
        return 1
    else:
        return 0

new_df['new_marriage'] = new_df['marriage'].apply(is_married)
new_df[['marriage', 'new_marriage']]

In [ ]:
#Company_size encoding(0, 1, others)
def compSize_0(x):
    if (x == 0):
        return 1
    else:
        return 0

def compSize_1(x):
    if(x == 1):
        return 1
    else:
        return 0

def compSize_else(x):
    if(x > 1):
        return 1
    else:
        return 0

new_df['compSize_0'] = new_df['company_size'].apply(compSize_0)
new_df['compSize_1'] = new_df['company_size'].apply(compSize_1)
new_df['compSize_else'] = new_df['company_size'].apply(compSize_else)

new_df[['company_size', 'compSize_0','compSize_1', 'compSize_else']]

In [ ]:
#Occupation encoding( )
def Manager(x):
    if (x//100 == 1):
        return 1
    else:
        return 0

def Expert(x):
    if(x//100 == 2):
        return 1
    else:
        return 0

def Office(x):
    if(x//100 == 3):
        return 1
    else:
        return 0

def Service(x):
    if(x//100 == 4):
        return 1
    else:
        return 0

def Sale(x):
    if(x//100 == 5):
        return 1
    else:
        return 0

def Skilled(x):
    if(x//100 == 6):
        return 1
    else:
        return 0

def Func(x):
    if(x//100  == 7):
        return 1
    else:
        return 0

def Operation(x):
    if(x//100 == 8):
        return 1
    else:
        return 0

def SimLabor(x):
    if(x//100 == 9):
        return 1
    else:
        return 0

def others(x):
    if(x//100 == 0 or x//100 >=10):
        return 1
    else:
        return 0

new_df['Manager'] = new_df['occupation'].apply(Manager)
new_df['Expert'] = new_df['occupation'].apply(Expert)
new_df['Office'] = new_df['occupation'].apply(Office)
new_df['Service'] = new_df['occupation'].apply(Service)
new_df['Sale'] = new_df['occupation'].apply(Sale)
new_df['Skilled'] = new_df['occupation'].apply(Skilled)
new_df['Func'] = new_df['occupation'].apply(Func)
new_df['Operation'] = new_df['occupation'].apply(Operation)
new_df['SimLabor'] = new_df['occupation'].apply(SimLabor)
new_df['others'] = new_df['occupation'].apply(others)


new_df[['occupation','Manager', 'Expert', 'Office','Service','Sale','Skilled','Func','Operation','SimLabor','others']]

In [ ]:
#gender encoding
def is_male(x):
    if(x == 1):
        return 1
    else:
        return 0

#기존 컬럼에 대체(남자면 1)
new_df['gender'] = new_df['gender'].apply(is_male)

In [ ]:
#reason_none_worker encoding(1 : not worker, 0: worker)
def not_working(x):
    if (x != 0): #실업자면 1, 취직자면 0
        return 1
    else:
        return 0


#기존 컬럼에 대체(실업자면 1)
new_df['reason_none_worker'] = new_df['reason_none_worker'].apply(not_working)
new_df['reason_none_worker']

In [ ]:
new_df.to_csv('new_df.csv')

In [ ]:
from math import sqrt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error



x = new_df.drop(columns=['income', 'occupation', 'marriage', 'region', 'company_size']) #독립변수
y = new_df['income'] #종속변수

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

rf_model = RandomForestRegressor(random_state = 42)

#그리드 서치를 위한 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators' : [100, 200, 300],
    'max_depth' : [None, 10, 20],
    'min_samples_split' : [2,5,10]
}

# 그리드 서치 수행

grid_search = GridSearchCV(estimator = rf_model, param_grid = param_grid, scoring = 'neg_mean_squared_error', cv = 5)
grid_search.fit(X_train, y_train)

best_rf_model = grid_search.best_estimator_

y_pred = best_rf_model.predict(X_test)
rmse = sqrt(mean_squared_error(y_test, y_pred))

print("Best parameters : ", grid_search.best_params_)
print(f"RMSE : {rmse}")

In [ ]:
from math import sqrt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

#income과의 상관계수의 절댓값이 0.2 이상인 변수들을 주요 변수들로 보고 분석
# -> family_member, gender, education_level, company_size, reason_none_worker, age9


x = new_df[['family_member', 'gender', 'education_level', 'company_size', 'reason_none_worker', 'age']] #독립변수
y = new_df['income'] #종속변수

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

rf_model = RandomForestRegressor(random_state = 42)

#그리드 서치를 위한 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators' : [100, 200, 300],
    'max_depth' : [None, 10, 20],
    'min_samples_split' : [2,5,10]
}

# 그리드 서치 수행

grid_search = GridSearchCV(estimator = rf_model, param_grid = param_grid, scoring = 'neg_mean_squared_error', cv = 5)
grid_search.fit(X_train, y_train)

best_rf_model = grid_search.best_estimator_

y_pred = best_rf_model.predict(X_test)
rmse = sqrt(mean_squared_error(y_test, y_pred))

print("Best parameters : ", grid_search.best_params_)
print(f"RMSE : {rmse}")